# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities.dropna(inplace=True, how='any')
cities


,Unnamed: 0,city,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
0,0,Arraial do Cabo,-22.9661,-42.0278,76.86,80.56,19.57,0,73
1,1,Matara,5.9485,80.5353,78.64,78.64,8.08,11,81
2,2,Prainha,-1.8000,-53.4800,88.61,88.61,2.68,57,54
3,3,Punta Arenas,-53.1500,-70.9167,33.91,39.22,37.98,90,93
4,4,Yaan,7.3833,8.5667,75.36,75.36,3.36,95,83
...,...,...,...,...,...,...,...,...,...
567,618,Kitimat,54.0524,-128.6534,41.92,43.18,1.34,100,30
568,619,Iquique,-20.2208,-70.1431,64.02,64.02,8.70,1,68
569,621,Ryōtsu-minato,38.0833,138.4333,56.07,56.07,8.10,81,64
570,622,Souillac,-20.5167,57.5167,75.36,75.36,10.49,4,80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Configure gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '275px',
    'height': '275px',
    'border': '1px solid black',
    'padding': '1px',
    'zoom_level': 12
}

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities[['latitude', 'longitude']]

humidity = cities['humidity']

# Add Heatmap layer to map
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

fig.add_layer(heat_layer)
plt.savefig("hum-heatmap.png")


print(fig)


Figure(layout=FigureLayout(border='1px solid black', height='275px', padding='1px', width='275px'))


<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
ideal_cities = cities.loc[(cities['max temperature'] < 80) & (cities['max temperature'] > 70) & (cities['humidity'] <30)]
ideal_cities

,Unnamed: 0,city,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
19,20,Copiapó,-27.3667,-70.3333,73.65,73.65,9.19,1,24
59,63,Abu Kamal,34.4506,40.9171,75.61,75.61,5.66,31,26
129,139,Kanye,-24.9667,25.3327,74.52,74.52,16.22,0,14
149,160,Jalu,29.0331,21.5482,78.48,78.48,8.52,1,28
252,270,San Juan,-31.5375,-68.5364,73.42,73.42,9.69,0,22
297,318,General Pico,-35.6566,-63.7568,73.02,73.02,17.54,5,17
368,399,Suruç,36.9761,38.4250,71.80,71.80,2.28,0,17
395,429,Sakakah,29.9697,40.2064,77.13,77.13,10.80,3,27
423,460,Gobabis,-22.4500,18.9667,78.64,78.64,12.08,0,6
456,496,Dutlwe,-23.9833,23.9000,78.93,78.93,19.57,0,13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_cities[['city', 'latitude', 'longitude']]
hotel_df['Hotel Name'] = ''
hotel_df.reset_index(drop=True)

<ipython-input-16-e6d4478e7f60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,city,latitude,longitude,Hotel Name
0,Copiapó,-27.3667,-70.3333,
1,Abu Kamal,34.4506,40.9171,
2,Kanye,-24.9667,25.3327,
3,Jalu,29.0331,21.5482,
4,San Juan,-31.5375,-68.5364,
5,General Pico,-35.6566,-63.7568,
6,Suruç,36.9761,38.4250,
7,Sakakah,29.9697,40.2064,
8,Gobabis,-22.4500,18.9667,
9,Dutlwe,-23.9833,23.9000,


In [23]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

param = {
    "location": f'37.6775, -113.0619',
    "radius": 5000,
    "keyword": 'hotel',
    "key": g_key
}

response = requests.get(base_url, params=params).json()
    
results = response['results']

In [29]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

param = {
    "radius": 5000,
    "keyword": 'lodging',
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    city = row['city']

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'
       

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    try:
        
        print(f'Getting result for {city}')
        # add the results to the dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# check the results
hotel_df

Getting result for Copiapó
Missing field/result... skipping.
Getting result for Abu Kamal
Missing field/result... skipping.
Getting result for Kanye
Missing field/result... skipping.
Getting result for Jalu
Missing field/result... skipping.
Getting result for San Juan
Missing field/result... skipping.
Getting result for General Pico
Missing field/result... skipping.
Getting result for Suruç
Missing field/result... skipping.
Getting result for Sakakah
Missing field/result... skipping.
Getting result for Gobabis
Missing field/result... skipping.
Getting result for Dutlwe
Missing field/result... skipping.
Getting result for Najrān
Missing field/result... skipping.
Getting result for Lubango
Missing field/result... skipping.


,city,latitude,longitude,Hotel Name
19,Copiapó,-27.3667,-70.3333,
59,Abu Kamal,34.4506,40.9171,
129,Kanye,-24.9667,25.3327,
149,Jalu,29.0331,21.5482,
252,San Juan,-31.5375,-68.5364,
297,General Pico,-35.6566,-63.7568,
368,Suruç,36.9761,38.4250,
395,Sakakah,29.9697,40.2064,
423,Gobabis,-22.4500,18.9667,
456,Dutlwe,-23.9833,23.9000,


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [31]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '275px',
    'height': '275px',
    'border': '1px solid black',
    'padding': '1px',
    'zoom_level': 12
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

plt.savefig("hotel_map.png")


print(fig)

Figure(layout=FigureLayout(border='1px solid black', height='275px', padding='1px', width='275px'))


<Figure size 432x288 with 0 Axes>